In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.optimizers import Adam
import numpy as np

# Load IMDB dataset
vocab_size = 5000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

# Preprocess data
max_words = 400
x_train = sequence.pad_sequences(x_train, maxlen=max_words)
x_test = sequence.pad_sequences(x_test, maxlen=max_words)

# Define LSTM model
lstm_model = Sequential(name="LSTM_Model")
lstm_model.add(Embedding(input_dim=vocab_size, output_dim=32, input_length=max_words))
lstm_model.add(LSTM(units=128, activation='tanh', return_sequences=False))
lstm_model.add(Dense(units=1, activation='sigmoid'))

# Compile model
optimizer = Adam(learning_rate=0.001)
lstm_model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train model
lstm_model.fit(x_train, y_train, batch_size=64, epochs=3, validation_split=0.2)

# Evaluate model
loss, accuracy = lstm_model.evaluate(x_test, y_test, verbose=0)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Decode IMDB reviews
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

# Predict sentiment for 3 test reviews
predictions = lstm_model.predict(x_test)

for i in range(3):
    review_text = ' '.join([reverse_word_index.get(index - 3, '[UNK]') for index in x_test[i] if index >= 3])
    sentiment_score = predictions[i][0]
    sentiment = "Positive" if sentiment_score > 0.5 else "Negative"

    print(f"\nReview {i+1}: {review_text}")
    print(f"Predicted Sentiment: {sentiment} (Confidence: {sentiment_score:.2f})")
